In [1]:
import pandas as pd
import pymysql

In [2]:
db_name = "premier_league"
db_host = "localhost"
db_username = "root"
db_password = "Garracrema25"

In [3]:
try:
    conn = pymysql.connect(host=db_host,
                            port=int(3306),
                            user='root',
                            passwd=db_password,
                            db=db_name)
except e:
    print(e)

In [4]:
if conn:
    print("Connection Successful!")
else:
    print("There was an error")

Connection Successful!


In [5]:
full_table = pd.read_sql_query("select * from matches_data limit 5", conn)
full_table

,Match_Name,Match_Date,Team,Manager,Captain,Score,Penalties,Fouls,Corners,Crosses,...,Possession,PassingAccuracy,SucPass,NumofPass,ShotsAccuracy,SucShots,NumofShots,SavesAccuracy,SucSaves,NumofSaves
0,Liverpool vs. Norwich City,8/9/2019 0:00:00,Liverpool,JÃ¼rgen Klopp,Jordan Henderson,4,0,10,11,15,...,0.57,0,409,521,0.47,7,15,0.83,5,6
1,Liverpool vs. Norwich City,8/9/2019 0:00:00,Norwich City,Daniel Farke,Grant Hanley,1,0,12,2,5,...,0.43,0,304,395,0.46,6,13,0.42,3,7
2,West Ham United vs. Manchester City,8/10/2019 0:00:00,West Ham United,Manuel Pellegrini,Aaron Cresswell,0,0,11,1,10,...,0.43,0,347,428,0.60,3,5,0.50,4,8
3,West Ham United vs. Manchester City,8/10/2019 0:00:00,Manchester City,Pep Guardiola,David Silva,5,0,15,1,8,...,0.57,0,495,564,0.62,8,13,1.00,3,3
4,Burnley vs. Southampton,8/10/2019 0:00:00,Burnley,Sean Dyche,Ben Mee,3,0,6,2,7,...,0.46,0,214,346,0.44,4,9,1.00,3,3


how many matches each team has played? as home? as visitor?
which team wins more at home? and as visitor, which one is the worse in both categories?
Rank them by each category
posession - analyze each category based on possession


In [6]:
number_of_games = pd.read_sql_query("""
select
count(distinct Match_Name) as number_of_matches
from matches_data
""", conn)

number_of_games.style.hide_index()

number_of_matches
380


In [7]:
number_of_teams = pd.read_sql_query("""
select
count(distinct Team) as count_of_teams
from matches_data
""", conn)

number_of_teams.style.hide_index()

count_of_teams
20


In [8]:
count_home_per_team= pd.read_sql_query("""
with home_visitor as
(
select
substr(Match_Name, 1, instr(Match_Name, ' vs. ')) as home,
substr(Match_Name, instr(Match_Name, '.') + 1) as visitor
from matches_data
)
select
home,
(count(*)/2) as games_at_home
from home_visitor
group by home
""", conn)

count_home_per_team.style.hide_index()

home,games_at_home
Liverpool,19.000000
West Ham United,19.000000
Burnley,19.000000
Watford,19.000000
Bournemouth,19.000000
Crystal Palace,19.000000
Tottenham Hotspur,19.000000
Newcastle United,19.000000
Leicester City,19.000000
Manchester United,19.000000


In [9]:
count_visits_per_team= pd.read_sql_query("""
with home_visitor as
(
select
substr(Match_Name, 1, instr(Match_Name, ' vs. ')) as home,
substr(Match_Name, instr(Match_Name, '.') + 1) as visitor
from matches_data
)
select
visitor,
(count(*)/2) as games_away
from home_visitor
group by visitor
""", conn)

count_visits_per_team.style.hide_index()

visitor,games_away
Norwich City,19.000000
Manchester City,19.000000
Southampton,19.000000
Brighton & Hove Albion,19.000000
Sheffield United,19.000000
Everton,19.000000
Aston Villa,19.000000
Arsenal,19.000000
Wolverhampton Wanderers,19.000000
Chelsea,19.000000


In [10]:
goals_per_team = pd.read_sql_query("""
select
Team, sum(Score) as total_goals
from matches_data
group by Team
order by total_goals desc
""", conn)

goals_per_team.style.hide_index()

Team,total_goals
Manchester City,102.000000
Liverpool,85.000000
Chelsea,69.000000
Leicester City,67.000000
Manchester United,66.000000
Tottenham Hotspur,61.000000
Arsenal,56.000000
Southampton,51.000000
Wolverhampton Wanderers,51.000000
West Ham United,49.000000


In [11]:
goals_at_home = pd.read_sql_query("""
select
substr(Match_Name, 1, instr(Match_Name, ' vs. ')) as home, sum(Score) as home_goals
from matches_data
group by home
order by home_goals desc
""", conn)

goals_at_home.style.hide_index()

home,home_goals
Manchester City,70.000000
Liverpool,68.000000
West Ham United,63.000000
Arsenal,60.000000
Manchester United,57.000000
Southampton,56.000000
Norwich City,56.000000
Tottenham Hotspur,53.000000
Bournemouth,52.000000
Leicester City,52.000000


In [12]:
goals_away = pd.read_sql_query("""
select
substr(Match_Name, instr(Match_Name, '. ') + 1) as away, sum(Score) as away_goals
from matches_data
group by away
order by away_goals desc
""", conn)

goals_away.style.hide_index()

away,away_goals
Chelsea,77.000000
Manchester City,67.000000
Aston Villa,56.000000
Leicester City,56.000000
Southampton,55.000000
Everton,55.000000
Newcastle United,55.000000
Tottenham Hotspur,55.000000
Bournemouth,53.000000
Watford,51.000000


In [14]:
fouls_made = pd.read_sql_query("""
select
Team,
sum(Fouls) as fouls_made
from matches_data
group by Team
order by fouls_made desc
""", conn)

fouls_made.style.hide_index()

Team,fouls_made
Watford,576.000000
Everton,537.000000
Aston Villa,527.000000
Southampton,526.000000
Burnley,503.000000
Crystal Palace,500.000000
Sheffield United,497.000000
Arsenal,497.000000
Tottenham Hotspur,495.000000
Manchester United,486.000000


In [15]:
fouls_made = pd.read_sql_query("""
select
Team,
avg(Fouls) as avg_fouls_made
from matches_data
group by Team
order by avg_fouls_made desc
""", conn)

fouls_made.style.hide_index()

Team,avg_fouls_made
Watford,15.157900
Everton,14.131600
Aston Villa,13.868400
Southampton,13.842100
Burnley,13.236800
Crystal Palace,13.157900
Sheffield United,13.078900
Arsenal,13.078900
Tottenham Hotspur,13.026300
Manchester United,12.789500


In [16]:
saves_made = pd.read_sql_query("""
select
Team,
sum(NumofSaves) as saves_made
from matches_data
group by Team
order by saves_made desc
""", conn)

saves_made.style.hide_index()

Team,saves_made
Norwich City,201.000000
Newcastle United,190.000000
Arsenal,189.000000
Bournemouth,182.000000
West Ham United,178.000000
Aston Villa,175.000000
Tottenham Hotspur,172.000000
Crystal Palace,169.000000
Watford,166.000000
Brighton & Hove Albion,164.000000


In [17]:
avg_saves_made = pd.read_sql_query("""
select
Team,
avg(NumofSaves) as avg_saves_made
from matches_data
group by Team
order by avg_saves_made desc
""", conn)

avg_saves_made.style.hide_index()

Team,avg_saves_made
Norwich City,5.289500
Newcastle United,5.000000
Arsenal,4.973700
Bournemouth,4.789500
West Ham United,4.684200
Aston Villa,4.605300
Tottenham Hotspur,4.526300
Crystal Palace,4.447400
Watford,4.368400
Brighton & Hove Albion,4.315800


In [18]:
avg_save_accuracy = pd.read_sql_query("""
select
Team,
avg(SavesAccuracy) as avg_saves_acccuracy
from matches_data
group by Team
order by avg_saves_acccuracy desc
""", conn)

avg_save_accuracy

,Team,avg_saves_acccuracy
0,Tottenham Hotspur,0.748684
1,Arsenal,0.743421
2,Crystal Palace,0.725000
3,Newcastle United,0.721053
4,Sheffield United,0.720526
5,Leicester City,0.703684
6,Manchester United,0.698421
7,West Ham United,0.696316
8,Burnley,0.691316
9,Brighton & Hove Albion,0.669737


In [19]:
total_shots_by_team = pd.read_sql_query("""
select
Team,
sum(NumofShots) as total_shots
from matches_data
group by team
order by total_shots desc
""", conn)

total_shots_by_team.style.hide_index()

Team,total_shots
Manchester City,730.000000
Chelsea,619.000000
Liverpool,585.000000
Leicester City,533.000000
Manchester United,528.000000
Southampton,497.000000
Everton,465.000000
Brighton & Hove Albion,456.000000
Aston Villa,453.000000
Wolverhampton Wanderers,453.000000


In [20]:
avg_shots_by_team = pd.read_sql_query("""
select
Team,
avg(NumofShots) as avg_shots
from matches_data
group by team
order by avg_shots desc
""", conn)

avg_shots_by_team.style.hide_index()

Team,avg_shots
Manchester City,19.210500
Chelsea,16.289500
Liverpool,15.394700
Leicester City,14.026300
Manchester United,13.894700
Southampton,13.078900
Everton,12.236800
Brighton & Hove Albion,12.000000
Aston Villa,11.921100
Wolverhampton Wanderers,11.921100


In [21]:
shot_accuracy = pd.read_sql_query("""
select
Team,
sum(Score) / sum(NumofShots) as shot_accuracy
from matches_data
group by Team
order by shot_accuracy desc
""", conn)

shot_accuracy

,Team,shot_accuracy
0,Liverpool,0.1453
1,Manchester City,0.1397
2,Arsenal,0.1397
3,Tottenham Hotspur,0.1390
4,Leicester City,0.1257
5,Manchester United,0.1250
6,West Ham United,0.1184
7,Wolverhampton Wanderers,0.1126
8,Burnley,0.1120
9,Chelsea,0.1115


In [23]:
avg_shot_accuracy = pd.read_sql_query("""
with accuracy as
(
select
Team,
Score / NumofShots as shot_accuracy
from matches_data
)
select
Team,
avg(shot_accuracy) as avg_accuracy
from accuracy
group by Team
order by avg_accuracy  desc
""", conn)

avg_shot_accuracy.style.hide_index()

Team,avg_accuracy
Tottenham Hotspur,0.167221
Arsenal,0.165261
Liverpool,0.158271
Manchester City,0.147774
Manchester United,0.135932
Leicester City,0.131092
Sheffield United,0.129689
Burnley,0.126613
Wolverhampton Wanderers,0.122889
Chelsea,0.119487


In [25]:
avg_posession_by_team = pd.read_sql_query("""
select
Team,
avg(Possession) as avg_possession
from matches_data
group by Team
order by avg_possession desc
""", conn)

avg_posession_by_team.style.hide_index()

Team,avg_possession
Manchester City,0.664737
Liverpool,0.628158
Chelsea,0.604474
Leicester City,0.569474
Manchester United,0.557105
Arsenal,0.536579
Brighton & Hove Albion,0.522105
Tottenham Hotspur,0.520789
Norwich City,0.491579
Everton,0.490263


In [28]:
tackles_by_team = pd.read_sql_query("""
select
Team,
sum(Tackles) as total_tackles
from matches_data
group by Team
order by total_tackles desc
""", conn)

tackles_by_team.style.hide_index()

Team,total_tackles
Southampton,790.000000
Leicester City,780.000000
Chelsea,740.000000
Everton,739.000000
Crystal Palace,733.000000
West Ham United,720.000000
Wolverhampton Wanderers,716.000000
Norwich City,715.000000
Tottenham Hotspur,708.000000
Watford,683.000000


In [29]:
avg_tackles_by_team = pd.read_sql_query("""
select
Team,
avg(Tackles) as avg_tackles
from matches_data
group by Team
order by avg_tackles desc
""", conn)

avg_tackles_by_team.style.hide_index()

Team,avg_tackles
Southampton,20.789500
Leicester City,20.526300
Chelsea,19.473700
Everton,19.447400
Crystal Palace,19.289500
West Ham United,18.947400
Wolverhampton Wanderers,18.842100
Norwich City,18.815800
Tottenham Hotspur,18.631600
Watford,17.973700


In [32]:
avg_interceptions_by_team = pd.read_sql_query("""
select
Team,
avg(Interceptions) as avg_interceptions
from matches_data
group by Team
order by avg_interceptions desc
""", conn)

avg_interceptions_by_team.style.hide_index()

Team,avg_interceptions
Watford,8.131600
Leicester City,8.131600
Manchester United,7.894700
Brighton & Hove Albion,7.868400
Southampton,7.763200
Newcastle United,7.684200
West Ham United,7.657900
Bournemouth,7.578900
Everton,7.342100
Crystal Palace,7.289500


In [33]:
avg_passes_by_team = pd.read_sql_query("""
select
Team,
avg(NumofPass) as avg_passes
from matches_data
group by Team
order by avg_passes desc
""", conn)

avg_passes_by_team.style.hide_index()

Team,avg_passes
Manchester City,728.210500
Liverpool,660.789500
Chelsea,645.526300
Manchester United,557.131600
Leicester City,556.657900
Arsenal,521.105300
Tottenham Hotspur,518.526300
Brighton & Hove Albion,514.394700
Norwich City,481.342100
Wolverhampton Wanderers,470.947400


In [36]:
passes_to_score_by_team = pd.read_sql_query("""
select
Team,
sum(NumofPass) / sum(Score) as passes_to_score
from matches_data
group by Team
order by passes_to_score desc
""", conn)

passes_to_score_by_team.style.hide_index()

Team,passes_to_score
Norwich City,703.500000
Crystal Palace,513.903200
Brighton & Hove Albion,501.205100
Watford,428.138900
Sheffield United,420.769200
Bournemouth,399.300000
Everton,390.386400
Aston Villa,374.439000
Newcastle United,373.868400
Chelsea,355.507200


how many passes are needed to score against each team? which teams suffers more goals against at home?

In [37]:
goals_received_home = pd.read_sql_query("""
select
Match_Name,
substr(Match_Name, 1, instr(Match_Name, ' vs. ')) as home,
substr(Match_Name, instr(Match_Name, '.') + 1) as visitor,
Team as team,
Score as score
from matches_data
""", conn)

goals_received_home

,Match_Name,home,visitor,team,score
0,Liverpool vs. Norwich City,Liverpool,Norwich City,Liverpool,4
1,Liverpool vs. Norwich City,Liverpool,Norwich City,Norwich City,1
2,West Ham United vs. Manchester City,West Ham United,Manchester City,West Ham United,0
3,West Ham United vs. Manchester City,West Ham United,Manchester City,Manchester City,5
4,Burnley vs. Southampton,Burnley,Southampton,Burnley,3
...,...,...,...,...,...
755,Manchester City vs. Norwich City,Manchester City,Norwich City,Norwich City,0
756,Leicester City vs. Manchester United,Leicester City,Manchester United,Leicester City,0
757,Leicester City vs. Manchester United,Leicester City,Manchester United,Manchester United,2
758,Newcastle United vs. Liverpool,Newcastle United,Liverpool,Newcastle United,1
